In [7]:
import sys
sys.path.append('/workspaces/asclepius/src/Api')

print(sys.path)

['/workspaces/asclepius/src/Api/notebooks', '/usr/local/lib/python311.zip', '/usr/local/lib/python3.11', '/usr/local/lib/python3.11/lib-dynload', '', '/home/vscode/.local/lib/python3.11/site-packages', '/usr/local/lib/python3.11/site-packages', '/workspaces/asclepius/src/Api', '/workspaces/asclepius/src/Api']


In [8]:
import semantic_kernel as sk
import pandas as pd

import Plugins.parse_notes_plugin as pnp

In [9]:
note_df = pd.read_csv('/workspaces/asclepius/src/Api/data/notes_no_PHI.csv')

note = note_df['NOTE_TEXT'][1]

print(note)

Depression Screen 
Positive:  PHQ-2 score >= 3 or PHQ-9 score >= 9
PHQ-2 Total: 0 (4/16/2024 11:35 AM)
PHQ-9 Total: 0 (4/16/2024 11:35 AM)
 
DEPRESSION PLAN OF CARE
Her depression screen was negative.
 
Chase Humdinger is a 83 y.o. female

Chief Complaint
Patient presents with
• Annual Wellness Visit (Medicare)
• Medication Review
  Patient has been taking half of all her medications, patient states her smart watch tells her that she has a fib
 
 
Pt is here for routine check up.
She has history of palpitations and states occasionally she will be notified by her watch that her pulse is high while she feels the palpitations. She has cut the cytomel in half and is taking over other day which she feels like has improved her symptoms. 
She's also been cutting her toprol in half and BP has been high. Discussed taking full pull routinely to help regulate pulse and improve BP. 
 
Reviewed previous labs 
History of elevated prolactin - pt didn't like the medication prescribed to lower prolacti

In [10]:
plugin = pnp.ParseNotesPlugin()
arguments = sk.KernelArguments(input=note)

result = plugin.get_chief_complaint(**arguments)

print(result)

Chief Complaint
Patient presents with
• Annual Wellness Visit (Medicare)
• Medication Review
  Patient has been taking half of all her medications, patient states her smart watch tells her that she has a fib
 
 
Pt is here for routine check up.
She has history of palpitations and states occasionally she will be notified by her watch that her pulse is high while she feels the palpitations. She has cut the cytomel in half and is taking over other day which she feels like has improved her symptoms. 
She's also been cutting her toprol in half and BP has been high. Discussed taking full pull routinely to help regulate pulse and improve BP. 
 
Reviewed previous labs 
History of elevated prolactin - pt didn't like the medication prescribed to lower prolactin. She's been taking a natural supplement and would like to recheck lab. 
 
 
Thyroid Disorder - advised to take medications 6 days per week after last lab. Pt has been taking 1/2 of cytomel every other day
 She is seen today for follow up 

In [11]:
ros_result = plugin.get_ros(**arguments)

print(ros_result)

ROS
Constitutional: Negative for chills, diaphoresis and unexpected weight change. 
HENT: Negative for hearing loss, nosebleeds and sore throat.  
Eyes: Negative for pain, discharge and redness. 
Respiratory: Negative for chest tightness, shortness of breath and wheezing.  
Cardiovascular: Negative.  
Gastrointestinal: Negative for abdominal pain, blood in stool, constipation, diarrhea and nausea. 
Genitourinary: Negative for difficulty urinating, dysuria and flank pain. 
Musculoskeletal: see HPI
Skin: Negative for rash. 
Neurological: Negative for dizziness, seizures and syncope. 
Psychiatric/Behavioral: Negative for behavioral problems and hallucinations. 
 




In [12]:
assessment_result = plugin.get_assessment(**arguments)

print(assessment_result)

ASSESSMENT and PLAN: 
 

  ICD-10-CM ICD-9-CM 
1. Medicare annual wellness visit, subsequent Z00.00 V70.0 
 
2. Hypothyroidism due to acquired atrophy of thyroid E03.4 244.8 liothyronine (CYTOMEL) 5 mcg Tablet
  246.8 TSH
  T3 FREE
  T4 FREE
  levothyroxine 137 mcg tablet
  T4 FREE
  T3 FREE
  TSH
 
3. Prolactinoma D35.2 227.3 PROLACTIN
  PROLACTIN
 
4. Vitamin D deficiency E55.9 268.9 VITAMIN D 25 HYDROXY
  VITAMIN D 25 HYDROXY
 
5. Essential hypertension, benign I10 401.1 CBC WITHOUT DIFFERENTIAL
  COMPREHENSIVE METABOLIC PANEL
  LIPID PANEL
  metoprolol succinate (TOPROL XL) 50 mg Extended Release 24 hour tablet
  LIPID PANEL
  COMPREHENSIVE METABOLIC PANEL
  CBC WITHOUT DIFFERENTIAL
 
6. Palpitations R00.2 785.1 metoprolol succinate (TOPROL XL) 50 mg Extended Release 24 hour tablet
  EKG 12-LEAD
 
7. Lumbar disc disease M51.9 722.93 traMADoL (ULTRAM) 50 mg tablet
 
 
 
Labs today
Increase toprol back to 1 pill daily
Decrease cytomel to 5mcg daily, continue levothyroxine
Tramadol re